*Instructions*
---------------
Before starting to work, please extract data from data_02.zip to a folder called **data** located at the same level that this notebook.

# 2) Loading and saving data

In [7]:
import pandas as pd
import numpy as np

## Loading CSV

In [3]:
input_path = '/home/dsc/Data/us_dot/traffic/'
filename = 'T100_SEGMENT_ALL_CARRIER_2015.zip'
file = input_path + filename



In [4]:
! ls $file

/home/dsc/Data/us_dot/traffic/T100_SEGMENT_ALL_CARRIER_2015.zip


In [5]:
! unzip -o $file

Archive:  /home/dsc/Data/us_dot/traffic/T100_SEGMENT_ALL_CARRIER_2015.zip
  inflating: 914310910_T_T100_SEGMENT_ALL_CARRIER_2015_All.csv  
  inflating: 914310910_T_T100_SEGMENT_ALL_CARRIER_ReadMe.csv  
  inflating: 914310910_T_T100_SEGMENT_ALL_CARRIER_Terms.csv  


In [6]:
notebook_file_path = '914310910_T_T100_SEGMENT_ALL_CARRIER_2015_All.csv'

In [7]:
! ls -lrt

total 50876
-rw-r--r-- 1 dsc dsc     4268 nov  8  2015 914310910_T_T100_SEGMENT_ALL_CARRIER_Terms.csv
-rw-r--r-- 1 dsc dsc     4341 nov  8  2015 914310910_T_T100_SEGMENT_ALL_CARRIER_ReadMe.csv
-rw-r--r-- 1 dsc dsc 51575427 nov  8  2015 914310910_T_T100_SEGMENT_ALL_CARRIER_2015_All.csv
-rw-r--r-- 1 dsc dsc   374081 ene 12 10:14 01-numpy_pandas_introduction-clean.ipynb
-rw-r--r-- 1 dsc dsc   115591 ene 12 10:42 02-loading_and_saving_data_2018_04_14-clean.ipynb


First, we need to prepare one csv file to read!

In [8]:
! ls ./data #not working on windows

ls: cannot access './data': No such file or directory


In [9]:
! unzip ./data/T100_SEGMENT_ALL_CARRIER_2015.zip #not working on windows

unzip:  cannot find or open ./data/T100_SEGMENT_ALL_CARRIER_2015.zip, ./data/T100_SEGMENT_ALL_CARRIER_2015.zip.zip or ./data/T100_SEGMENT_ALL_CARRIER_2015.zip.ZIP.


In [10]:
! ls -lrt ./

total 50876
-rw-r--r-- 1 dsc dsc     4268 nov  8  2015 914310910_T_T100_SEGMENT_ALL_CARRIER_Terms.csv
-rw-r--r-- 1 dsc dsc     4341 nov  8  2015 914310910_T_T100_SEGMENT_ALL_CARRIER_ReadMe.csv
-rw-r--r-- 1 dsc dsc 51575427 nov  8  2015 914310910_T_T100_SEGMENT_ALL_CARRIER_2015_All.csv
-rw-r--r-- 1 dsc dsc   374081 ene 12 10:14 01-numpy_pandas_introduction-clean.ipynb
-rw-r--r-- 1 dsc dsc   115591 ene 12 10:42 02-loading_and_saving_data_2018_04_14-clean.ipynb


In [111]:
#Defining path to CSV file
my_file_path='914310910_T_T100_SEGMENT_ALL_CARRIER_2015_All.csv'

In [112]:
#Reading file and saving in df
df = pd.read_csv(my_file_path)
df.head(10)

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,...,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50
0,0.0,1.0,528.0,11.0,2.0,0.0,0.0,3899.0,0.0,0.0,...,6,669,1,2015,1,1,8,L,IF,NaN
1,0.0,2.0,3621.0,22.0,2.0,0.0,0.0,2589.0,0.0,0.0,...,6,669,1,2015,1,1,6,L,IF,NaN
2,0.0,1.0,2169.0,11.0,2.0,0.0,0.0,1210.0,0.0,0.0,...,6,669,1,2015,1,1,3,L,DF,NaN
3,0.0,1.0,1342.0,11.0,1.0,0.0,0.0,2316.0,0.0,0.0,...,6,669,1,2015,1,1,5,L,IF,NaN
4,0.0,1.0,275.0,11.0,1.0,0.0,0.0,1023.0,0.0,0.0,...,6,669,1,2015,1,1,3,L,DF,NaN
5,0.0,1.0,264.0,11.0,1.0,0.0,0.0,3934.0,0.0,0.0,...,6,669,1,2015,1,1,8,L,IF,NaN
6,0.0,1.0,5619.0,12.0,1.0,0.0,0.0,3678.0,0.0,0.0,...,6,658,1,2015,1,1,8,L,IF,NaN
7,0.0,1.0,5150.0,14.0,5.0,0.0,0.0,5341.0,0.0,0.0,...,6,658,1,2015,1,1,11,L,IF,NaN
8,0.0,1.0,5619.0,12.0,1.0,0.0,0.0,3678.0,0.0,0.0,...,6,658,1,2015,1,1,8,L,IF,NaN
9,0.0,1.0,5150.0,14.0,6.0,0.0,0.0,93.0,0.0,0.0,...,6,658,1,2015,1,1,1,L,DF,NaN


In [13]:
df.describe()

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,...,DEST_STATE_FIPS,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,Unnamed: 50
count,146938.000000,146938.000000,1.469380e+05,146938.000000,146938.000000,1.469380e+05,146938.000000,146938.000000,146938.000000,146938.000000,...,133000.000000,146938.000000,146938.000000,146938.000000,146938.000000,146938.0,146938.000000,146938.000000,146938.000000,0.0
mean,20.091270,22.263002,7.817593e+05,2385.648035,1899.964339,9.782733e+04,2932.011161,989.309096,2806.873573,2339.063170,...,24.509098,85.621854,5.500252,595.634410,1.176585,2015.0,1.251739,2.504043,2.530169,NaN
std,36.850212,35.603689,1.400154e+06,4500.311182,3673.630008,5.223834e+05,19062.246309,1185.712296,5049.683558,4391.945531,...,17.847929,158.078243,1.576414,161.396262,0.458067,0.0,0.434013,1.121343,2.347791,NaN
min,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,10.000000,1.000000,2015.0,1.000000,1.000000,1.000000,NaN
25%,0.000000,1.000000,3.500000e+04,50.000000,11.000000,0.000000e+00,0.000000,272.000000,114.000000,90.000000,...,8.000000,23.000000,6.000000,614.000000,1.000000,2015.0,1.000000,1.000000,1.000000,NaN
50%,4.000000,8.000000,2.179555e+05,450.000000,312.000000,0.000000e+00,0.000000,636.000000,670.000000,538.000000,...,22.000000,43.000000,6.000000,629.000000,1.000000,2015.0,1.000000,3.000000,2.000000,NaN
75%,27.000000,29.000000,9.908450e+05,3066.000000,2380.000000,3.718750e+03,0.000000,1189.000000,3527.000000,2791.000000,...,39.000000,82.000000,6.000000,677.000000,1.000000,2015.0,2.000000,4.000000,3.000000,NaN
max,822.000000,815.000000,4.052840e+07,98382.000000,75216.000000,3.512707e+07,914525.000000,8569.000000,138379.000000,124124.000000,...,78.000000,961.000000,8.000000,890.000000,4.000000,2015.0,2.000000,4.000000,18.000000,NaN


In [14]:
! cat '914310910_T_T100_SEGMENT_ALL_CARRIER_ReadMe.csv'

"SYS_FIELD_NAME"," FIELD_DESC",
"YEAR","Year",
"QUARTER","Quarter (1-4)",
"MONTH","Month",
"ORIGIN_AIRPORT_ID","Origin Airport, Airport ID. An identification number assigned by US DOT to identify a unique airport.  Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.",
"ORIGIN_AIRPORT_SEQ_ID","Origin Airport, Airport Sequence ID. An identification number assigned by US DOT to identify a unique airport at a given point of time.  Airport attributes, such as airport name or coordinates, may change over time.",
"ORIGIN_CITY_MARKET_ID","Origin Airport, City Market ID. City Market ID is an identification number assigned by US DOT to identify a city market.  Use this field to consolidate airports serving the same city market.",
"ORIGIN","Origin Airport Code",
"ORIGIN_CITY_NAME","Origin Airport, City Name",
"ORIGIN_STATE_ABR","Origin Airport, State",
"ORIGIN_STATE_FIPS","Origin Airport, State FIPS Code (U.S.

In [15]:
#df?

In [16]:
#Number of rows on Dataframe
len(df)

146938

Make some operations with the df

In [17]:
df[df['DEPARTURES_PERFORMED'] > 50].head(10)

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,...,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50
37,0.0,61.0,5365411.0,18353.0,16933.0,571685.0,0.0,4395.0,0.0,0.0,...,6,627,1,2015,1,1,9,F,IF,NaN
42,0.0,61.0,5365411.0,18353.0,16827.0,1274568.0,0.0,4395.0,0.0,0.0,...,6,627,1,2015,1,1,9,F,IF,NaN
158,0.0,51.0,2706103.0,17384.0,13517.0,0.0,0.0,2002.0,0.0,0.0,...,6,627,1,2015,1,1,5,F,IF,NaN
159,0.0,51.0,2751949.0,17384.0,13746.0,0.0,0.0,2002.0,0.0,0.0,...,6,627,1,2015,1,1,5,F,IF,NaN
181,0.0,62.0,9292527.0,20584.0,16739.0,1505486.0,0.0,6504.0,0.0,0.0,...,6,637,1,2015,1,1,14,F,IF,NaN
186,0.0,62.0,9256991.0,20584.0,19123.0,1847391.0,0.0,6504.0,0.0,0.0,...,6,637,1,2015,1,1,14,F,IF,NaN
199,0.0,184.0,25907200.0,0.0,0.0,23870728.0,0.0,1506.0,0.0,0.0,...,6,696,2,2015,1,1,4,G,IF,NaN
201,0.0,62.0,8729600.0,0.0,0.0,7389870.0,0.0,1388.0,0.0,0.0,...,6,696,2,2015,1,1,3,G,IF,NaN
205,0.0,134.0,18867200.0,0.0,0.0,9486624.0,0.0,1506.0,0.0,0.0,...,6,696,2,2015,1,1,4,G,IF,NaN
212,0.0,59.0,4517040.0,17818.0,14384.0,309318.0,0.0,4661.0,0.0,0.0,...,6,696,1,2015,1,1,10,F,IF,NaN


In [18]:
df[df['DEPARTURES_PERFORMED'] > 50]['PAYLOAD'].head(10)

37      5365411.0
42      5365411.0
158     2706103.0
159     2751949.0
181     9292527.0
186     9256991.0
199    25907200.0
201     8729600.0
205    18867200.0
212     4517040.0
Name: PAYLOAD, dtype: float64

In [19]:
df[df['DEPARTURES_PERFORMED'] > 50]['PAYLOAD'].sum()

50535573064.0

In [20]:
%%timeit
df[df['DEPARTURES_PERFORMED'] > 50]['PAYLOAD'].sum()

22.7 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
len(df[df['DEPARTURES_PERFORMED'] > 50])

In order to download a file using Python, the use of requests module is recommended

In [ ]:
#!wget -o http://datos.madrid.es/egob/catalogo/207831-0-accidentes-trafico.xls

### Example:

In [21]:
import requests
#url="http://datos.madrid.es/egob/catalogo/210104-180-multas-circulacion-detalle.csv"
url = 'http://climatedataapi.worldbank.org/climateweb/rest/v1/country/cru/tas/year/CAN.csv'
response = requests.get(url)
if response.status_code != 200:
    print('Failed to get data:', response.status_code)
else:
    print('First 100 characters of data are')
    print(response.text[:100])

First 100 characters of data are
year,data
1901,-7.67241907119751
1902,-7.862711429595947
1903,-7.910782814025879
1904,-8.15572929382


In [22]:
import requests
my_path="http://datos.madrid.es/egob/catalogo/210104-180-multas-circulacion-detalle.csv"
multas_response = requests.get(my_path)

In [16]:
! wget 'http://datos.madrid.es/egob/catalogo/210104-180-multas-circulacion-detalle.csv'

--2019-01-12 11:34:17--  http://datos.madrid.es/egob/catalogo/210104-180-multas-circulacion-detalle.csv
Resolving datos.madrid.es (datos.madrid.es)... 149.235.17.74
Connecting to datos.madrid.es (datos.madrid.es)|149.235.17.74|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://datos.madrid.es/egob/catalogo/210104-180-multas-circulacion-detalle.csv [following]
--2019-01-12 11:34:17--  https://datos.madrid.es/egob/catalogo/210104-180-multas-circulacion-detalle.csv
Connecting to datos.madrid.es (datos.madrid.es)|149.235.17.74|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://datos.madrid.es:443/datosabiertos/MULTAS/2017/12/201712_detalle.csv [following]
--2019-01-12 11:34:17--  https://datos.madrid.es/datosabiertos/MULTAS/2017/12/201712_detalle.csv
Connecting to datos.madrid.es (datos.madrid.es)|149.235.17.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53252030 (51M) [text/csv]
Saving to:

In [2]:
multas_response

NameError: name 'multas_response' is not defined

In [21]:

multas_path = '210104-180-multas-circulacion-detalle.csv'
#multas_file = open(multas_path, 'wb')
#multas_file.write(multas_response.content)
#multas_file.close()

In [22]:
! wc -l {multas_path} #unix

204816 210104-180-multas-circulacion-detalle.csv


In [10]:
%pwd

'/home/dsc/Repos/numpy-pandas-notebook/Solved'

In [23]:
! head -n 5 {multas_path} #unix

Con el encoding latin-1 nos ahorramos problemas con caracteres acentuados.

In [24]:
df_multas = pd.read_csv(multas_path, sep=';', encoding='latin-1')
df_multas.head(20)


,CALIFICACION,LUGAR,MES,ANIO,HORA,IMP_BOL,DESCUENTO,PUNTOS,DENUNCIANTE,HECHO-BOL,VEL_LIMITE,VEL_CIRCULA,COORDENADA_X,COORDENADA_Y
0,LEVE,LUIS DE HOYOS SAINZ FTE 196,12,2017,13.00,90.0,SI,0,POLICIA MUNICIPAL,ESTACIONAR EN LUGAR PROHIBIDO DEBIDAMENTE SEÑA...,,,,
1,LEVE,AEROPUERTO SALIDA T 1,12,2017,12.10,90.0,SI,0,POLICIA MUNICIPAL,ESTACIONAR EN LUGAR PROHIBIDO DEBIDAMENTE SEÑA...,,,,
2,GRAVE,CTRA CANILLAS 2,12,2017,11.30,200.0,SI,0,POLICIA MUNICIPAL,ESTACIONAR EN ZONA SEÑALIZADA PARA USO EXCLUSI...,,,,
3,GRAVE,"M 30 NL KM 13,250 CARRIL 3",12,2017,6.21,500.0,SI,6,POLICIA MUNICIPAL,SOBREPASAR LA VELOCIDADMÁXIMA EN VÍAS LIMITADA...,70,131,,
4,LEVE,CL ORENSE 24,12,2017,17.42,90.0,SI,0,SER,"ESTACIONAR, SIN LA CORRESPONDIENTE AUTORIZACIÓ...",,,,
5,GRAVE,PO GENERAL SAGARDIA RAMOS 16,12,2017,7.08,200.0,SI,0,POLICIA MUNICIPAL,ESTACIONAR OBSTACULIZANDO LA UTILIZACIÓN DE UN...,,,,
6,LEVE,AEROPUERTO SALIDA T 1,12,2017,12.15,90.0,SI,0,POLICIA MUNICIPAL,ESTACIONAR EN LUGAR PROHIBIDO DEBIDAMENTE SEÑA...,,,,
7,GRAVE,CL ORENSE 8,12,2017,18.08,200.0,SI,0,SER,ESTACIONAR EN CARRIL DE CIRCULACION. ...,,,,
8,GRAVE,AV BADAJOZ 10,12,2017,3.45,200.0,SI,0,POLICIA MUNICIPAL,CONDUCCION NEGLIGENTE: CIRCULABA DE DERECHA A ...,,,,
9,GRAVE,VILLABLANCA 29,12,2017,20.21,200.0,SI,0,POLICIA MUNICIPAL,ESTACIONAR EN ZONA SEÑALIZADA PARA USO EXCLUSI...,,,,


In [38]:
df_multas['CALIFICACION'].unique()

array(['LEVE      ', 'GRAVE     ', 'MUY GRAVE '], dtype=object)

In [47]:
df_multas[df_multas['CALIFICACION'].str.strip() ==  'MUY GRAVE']

,CALIFICACION,LUGAR,MES,ANIO,HORA,IMP_BOL,DESCUENTO,PUNTOS,DENUNCIANTE,HECHO-BOL,VEL_LIMITE,VEL_CIRCULA,COORDENADA_X,COORDENADA_Y
65717,MUY GRAVE,M 11 PO CASTELLANA,12,2017,8.45,1000.0,SI,6,POLICIA MUNICIPAL,INCUMPLIR LA OBLIGACIÓNDE SOMETERSE A LAS PRUE...,,,,
66324,MUY GRAVE,HARTZENBUSCH-CARD. CISNEROS,12,2017,11.26,500.0,SI,6,AGENTES DE MOVILIDAD,CIRCULAR EN SENTIDO CONTRARIO AL AUTORIZADO. ...,,,,
66596,MUY GRAVE,ISAAC PERAL 42,12,2017,11.49,500.0,SI,6,AGENTES DE MOVILIDAD,CIRCULAR EN SENTIDO CONTRARIO AL AUTORIZADO. ...,,,,
66717,MUY GRAVE,GRAN VIA 63,12,2017,12.05,500.0,SI,6,AGENTES DE MOVILIDAD,CIRCULAR EN SENTIDO CONTRARIO AL AUTORIZADO. ...,,,,
70365,MUY GRAVE,PO DELICIAS 54,12,2017,9.17,500.0,SI,6,AGENTES DE MOVILIDAD,CIRCULAR EN SENTIDO CONTRARIO AL AUTORIZADO. ...,,,,
71558,MUY GRAVE,MARQ.VALDEIGL. 4,12,2017,16.42,500.0,SI,6,AGENTES DE MOVILIDAD,CIRCULAR EN SENTIDO CONTRARIO AL AUTORIZADO. ...,,,,
71966,MUY GRAVE,PRINCI.VERGARA 253,12,2017,8.51,1000.0,SI,6,AGENTES DE MOVILIDAD,CONDUCIR CON UNA TASA DE ALCOHOLEMIA NO PERMIT...,,,,
72700,MUY GRAVE,PZ ESPAÑA 1,12,2017,13.15,1000.0,SI,6,AGENTES DE MOVILIDAD,CONDUCIR CON PRESENCIA DE DROGAS EN EL ORGANIS...,,,,
72730,MUY GRAVE,RIB.CURTIDORES-FRAY C GLEZ,12,2017,13.27,500.0,SI,6,AGENTES DE MOVILIDAD,CIRCULAR EN SENTIDO CONTRARIO AL AUTORIZADO. ...,,,,
73307,MUY GRAVE,BAILEN-MAYOR,12,2017,16.19,500.0,SI,6,AGENTES DE MOVILIDAD,CONDUCCION TEMERARIA ...,,,,


Strings can be treaky...

In [29]:

df_multas[df_multas['LUGAR'] == ' LUIS DE HOYOS SAINZ FTE 196 ']

,CALIFICACION,LUGAR,MES,ANIO,HORA,IMP_BOL,DESCUENTO,PUNTOS,DENUNCIANTE,HECHO-BOL,VEL_LIMITE,VEL_CIRCULA,COORDENADA_X,COORDENADA_Y


but with a bit of care...

In [30]:
mi_portal ="LUIS DE HOYOS SAINZ FTE 196".strip()

In [31]:
df_multas[df_multas['LUGAR'].str.strip() == mi_portal]

,CALIFICACION,LUGAR,MES,ANIO,HORA,IMP_BOL,DESCUENTO,PUNTOS,DENUNCIANTE,HECHO-BOL,VEL_LIMITE,VEL_CIRCULA,COORDENADA_X,COORDENADA_Y
0,LEVE,LUIS DE HOYOS SAINZ FTE 196,12,2017,13.0,90.0,SI,0,POLICIA MUNICIPAL,ESTACIONAR EN LUGAR PROHIBIDO DEBIDAMENTE SEÑA...,,,,


We can play with this dataframe... XD

In [50]:
df_multas[df_multas['LUGAR'].str.contains("FERNANDEZ DE LOS RIOS 12")]

,CALIFICACION,LUGAR,MES,ANIO,HORA,IMP_BOL,DESCUENTO,PUNTOS,DENUNCIANTE,HECHO-BOL,VEL_LIMITE,VEL_CIRCULA,COORDENADA_X,COORDENADA_Y
11105,LEVE,CL FERNANDEZ DE LOS RIOS 12,12,2017,14.30,90.0,SI,0,SER,ESTACIONAR EN LUGAR PROHIBIDO DEBIDAMENTE SEÑA...,,,,
11108,LEVE,CL FERNANDEZ DE LOS RIOS 12,12,2017,14.52,90.0,SI,0,SER,ESTACIONAR EN LUGAR PROHIBIDO DEBIDAMENTE SEÑA...,,,,
13271,LEVE,CL FERNANDEZ DE LOS RIOS 12,12,2017,13.12,90.0,SI,0,SER,ESTACIONAR EN LUGAR PROHIBIDO DEBIDAMENTE SEÑA...,,,,
64688,LEVE,CL FERNANDEZ DE LOS RIOS 12,12,2017,17.26,90.0,SI,0,SER,ESTACIONAR EN LUGAR PROHIBIDO DEBIDAMENTE SEÑA...,,,,
113458,LEVE,CL FERNANDEZ DE LOS RIOS 12,12,2017,13.25,90.0,SI,0,SER,ESTACIONAR EN LUGAR PROHIBIDO DEBIDAMENTE SEÑA...,,,,
123471,LEVE,CL FERNANDEZ DE LOS RIOS 12,12,2017,18.23,90.0,SI,0,SER,ESTACIONAR EN LUGAR PROHIBIDO DEBIDAMENTE SEÑA...,,,,


In [33]:
df_multas['IMP_BOL'].mean()

104.16942118497181

In [34]:
df_multas['IMP_BOL'].sum()

21335460.0

In [51]:
descuentos = df_multas['DESCUENTO']
descuentos.value_counts()

SI    204814
NO         1
Name: DESCUENTO, dtype: int64

In [55]:
df_multas[df_multas['DESCUENTO'] =='NO']

,CALIFICACION,LUGAR,MES,ANIO,HORA,IMP_BOL,DESCUENTO,PUNTOS,DENUNCIANTE,HECHO-BOL,VEL_LIMITE,VEL_CIRCULA,COORDENADA_X,COORDENADA_Y
188166,GRAVE,EMBAJADORES-AV PLANETARIO,12,2017,13.34,300.0,NO,0,POLICIA MUNICIPAL,CIRCULAR TRANSPORTANDO MERCANCÍAS PELIGROSAS S...,,,,


In [57]:
maxima_multa = df_multas['IMP_BOL'].max()
maxima_multa

1000.0

In [59]:
df_multas[df_multas['IMP_BOL'] == maxima_multa]['HECHO-BOL'].unique()

array(['INCUMPLIR LA OBLIGACIÓNDE SOMETERSE A LAS PRUEBAS DE DETECCIÓN DE ALCOHOL O DE PRESENCIADE DROGAS EN EL ORGANISMO.           ',
       'CONDUCIR CON UNA TASA DE ALCOHOLEMIA NO PERMITIDA SUPERIOR A 0.50 MG/L EN AIRE ESPIRADO (0.30 MG/L NOVELES/PROF.)            ',
       'CONDUCIR CON PRESENCIA DE DROGAS EN EL ORGANISMO.                                                                            ',
       'CONDUCIR CON UNA TASA DE ALCOHOLEMIA NO PERMITIDA DE HASTA 0.50 MG/L EN AIRE ESPIRADO,CON ANTECEDENTES ART. 80.2.A) LSV.     '],
      dtype=object)

#### Exercise

Calculate the amount that the city hall took in, after discount. Assume that the discount is 50%, for each line that has 'SI' in column 'DESCUENTO'.

OJO: Un booleano se puede multiplicar por un número, actúa como 0 y 1. 

In [64]:
df_multas['DESCUENTO_PCT'] = 0.5 *(df_multas['DESCUENTO'] == 'SI')
df_multas['DESCUENTO_PCT'].head()

0    0.5
1    0.5
2    0.5
3    0.5
4    0.5
Name: DESCUENTO_PCT, dtype: float64

In [66]:
df_multas['DESCUENTO_PCT'].unique()

array([0.5, 0. ])

In [74]:
df_multas['IMPORTE_FINAL'] = df_multas['IMP_BOL'] * (1 - df_multas['DESCUENTO_PCT'])

Calcular el importe total recibido por cada tipo de infracción

In [75]:
df_multas.groupby('CALIFICACION')['IMPORTE_FINAL'].sum()

CALIFICACION
GRAVE         4718050.0
LEVE          5632080.0
MUY GRAVE      317750.0
Name: IMPORTE_FINAL, dtype: float64

Describir las diferentes infracciones que conllevan quitar puntos

In [88]:
df_multas[df_multas['PUNTOS'] >0]['HECHO-BOL'].unique()

array(['SOBREPASAR LA VELOCIDADMÁXIMA EN VÍAS LIMITADAS EN 60 km/h O MÁS.                                                            ',
       'EFECTUAR UN CAMBIO DE SENTIDO PROHIBIDO.                                                                                     ',
       'REBASAR UN SEMÁFORO EN FASE ROJA.                                                                                            ',
       'SOBREPASAR LA VELOCIDAD MEDIA EN TRAMO CONTROLADO.                                                                           ',
       'SOBREPASAR LA VELOCIDADMÁXIMA EN VÍAS LIMITADAS HASTA 50km/h.                                                                ',
       'UTILIZAR MANUALMENTE PANTALLAS, NAVEGADORES, DISPOSITIVO DE TELEFONIA, SISTEMA DE COMUNICACION O DE DETECCION DE RADAR       ',
       'CONDUCIR UTILIZANDO AURICULARES U OTROS DISPOSITIVOS QUE DISMINUYAN LA OBLIGATORIA ATENCION PERMANENTE A LA CONDUCCION       ',
       'NO OBEDECER LAS ÓRDENES DEL AGENTE QUE R

Sacar lugar con más multas

In [105]:
grouped = df_multas.groupby('LUGAR');
grouped.size()



LUGAR
 3                                             1
 S/N                                           1
 S/N CR CARABANCHEL VILLA                     69
 S/N CR VILLAVERDE VALLEC                     60
0 117                                          1
0 165                                          1
0 177                                          1
0 179                                          1
1 SUR 10 E                                     1
1 SUR-1 SUR                                    3
1095 23                                        2
12 DE OCTUBRE S/N                              2
A 1 M 30 CALZADA 2                             1
A 2 M 30                                       1
A 3 AV GRAN VIA DEL SURESTE                    1
A 3 KM 10                                      1
A 3 LATERAL M 40 SUR                           1
A 3 VIA SERVICIO BOSCO                         2
A 4 KM 5                                       1
A 42 PZ FERNANDEZ LADREDA                      1
A 5 CARLINA   

#### Let's time it

In [107]:
df_multas = pd.read_csv(multas_path, sep=';', encoding='latin-1')

In [108]:
%%timeit
df_multas['DESCUENTO_PCT'] = .5 *(df_multas['DESCUENTO'] == 'SI')
df_multas['IMPORTE_FINAL'] = df_multas['IMP_BOL'] * (1 - df_multas['DESCUENTO_PCT'])

90.3 ms ± 1.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Saving to Excel

In order to save and read excels, pandas needs addtional modules: xlrd, xlwt and xlsxwriter or openpyxl

In [113]:
df[df['DEPARTURES_PERFORMED'] > 50].to_excel('out.xls')

In [ ]:
pd.ExcelWriter?

In [114]:
df[df['DEPARTURES_PERFORMED'] > 50].to_excel('out.xlsx')

# Saving to CSV

In [ ]:
df.to_csv('out.csv')

If we don't pass any argument, to_csv will return the contents of the DataFrame, in csv format, as a string.

In [ ]:
file_as_string = df.to_csv()

file_as_string[:100]

# To Sql Database

In [116]:
import sqlite3
conn = sqlite3.connect('example.db')

In [117]:
df.to_sql('traffic', conn)

/home/dsc/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


# To dictionary and to json

json is the most widely used data interchange format over the web.

In [ ]:
!pwd #only for unix

In [118]:
df[:3].to_dict()

{'AIRCRAFT_CONFIG': {0: 1, 1: 1, 2: 1},
 'AIRCRAFT_GROUP': {0: 6, 1: 6, 2: 6},
 'AIRCRAFT_TYPE': {0: 669, 1: 669, 2: 669},
 'AIRLINE_ID': {0: 21407.0, 1: 21407.0, 2: 21407.0},
 'AIR_TIME': {0: 0.0, 1: 0.0, 2: 0.0},
 'CARRIER': {0: '10Q', 1: '10Q', 2: '10Q'},
 'CARRIER_GROUP': {0: 0, 1: 0, 2: 0},
 'CARRIER_GROUP_NEW': {0: 0.0, 1: 0.0, 2: 0.0},
 'CARRIER_NAME': {0: 'Swiss Air Ambulance',
  1: 'Swiss Air Ambulance',
  2: 'Swiss Air Ambulance'},
 'CLASS': {0: 'L', 1: 'L', 2: 'L'},
 'DATA_SOURCE': {0: 'IF', 1: 'IF', 2: 'DF'},
 'DEPARTURES_PERFORMED': {0: 1.0, 1: 2.0, 2: 1.0},
 'DEPARTURES_SCHEDULED': {0: 0.0, 1: 0.0, 2: 0.0},
 'DEST': {0: 'BSL', 1: 'TEB', 2: 'MFE'},
 'DEST_AIRPORT_ID': {0: 10762, 1: 15167, 2: 13256},
 'DEST_AIRPORT_SEQ_ID': {0: 1076203, 1: 1516702, 2: 1325602},
 'DEST_CITY_MARKET_ID': {0: 30762, 1: 35167, 2: 33256},
 'DEST_CITY_NAME': {0: 'Mulhouse, France',
  1: 'Teterboro, NJ',
  2: 'Mission/McAllen/Edinburg, TX'},
 'DEST_COUNTRY': {0: 'FR', 1: 'US', 2: 'US'},
 'DEST_COUN

In [119]:
df[:3].to_dict(orient='rows')

[{'AIRCRAFT_CONFIG': 1,
  'AIRCRAFT_GROUP': 6,
  'AIRCRAFT_TYPE': 669,
  'AIRLINE_ID': 21407.0,
  'AIR_TIME': 0.0,
  'CARRIER': '10Q',
  'CARRIER_GROUP': 0,
  'CARRIER_GROUP_NEW': 0.0,
  'CARRIER_NAME': 'Swiss Air Ambulance',
  'CLASS': 'L',
  'DATA_SOURCE': 'IF',
  'DEPARTURES_PERFORMED': 1.0,
  'DEPARTURES_SCHEDULED': 0.0,
  'DEST': 'BSL',
  'DEST_AIRPORT_ID': 10762,
  'DEST_AIRPORT_SEQ_ID': 1076203,
  'DEST_CITY_MARKET_ID': 30762,
  'DEST_CITY_NAME': 'Mulhouse, France',
  'DEST_COUNTRY': 'FR',
  'DEST_COUNTRY_NAME': 'France',
  'DEST_STATE_ABR': nan,
  'DEST_STATE_FIPS': nan,
  'DEST_STATE_NM': nan,
  'DEST_WAC': 427,
  'DISTANCE': 3899.0,
  'DISTANCE_GROUP': 8,
  'FREIGHT': 0.0,
  'MAIL': 0.0,
  'MONTH': 1,
  'ORIGIN': 'EWR',
  'ORIGIN_AIRPORT_ID': 11618,
  'ORIGIN_AIRPORT_SEQ_ID': 1161802,
  'ORIGIN_CITY_MARKET_ID': 31703,
  'ORIGIN_CITY_NAME': 'Newark, NJ',
  'ORIGIN_COUNTRY': 'US',
  'ORIGIN_COUNTRY_NAME': 'United States',
  'ORIGIN_STATE_ABR': 'NJ',
  'ORIGIN_STATE_FIPS': 34.0,

In [115]:
df[:3].to_json()

'{"DEPARTURES_SCHEDULED":{"0":0.0,"1":0.0,"2":0.0},"DEPARTURES_PERFORMED":{"0":1.0,"1":2.0,"2":1.0},"PAYLOAD":{"0":528.0,"1":3621.0,"2":2169.0},"SEATS":{"0":11.0,"1":22.0,"2":11.0},"PASSENGERS":{"0":2.0,"1":2.0,"2":2.0},"FREIGHT":{"0":0.0,"1":0.0,"2":0.0},"MAIL":{"0":0.0,"1":0.0,"2":0.0},"DISTANCE":{"0":3899.0,"1":2589.0,"2":1210.0},"RAMP_TO_RAMP":{"0":0.0,"1":0.0,"2":0.0},"AIR_TIME":{"0":0.0,"1":0.0,"2":0.0},"UNIQUE_CARRIER":{"0":"10Q","1":"10Q","2":"10Q"},"AIRLINE_ID":{"0":21407.0,"1":21407.0,"2":21407.0},"UNIQUE_CARRIER_NAME":{"0":"Swiss Air Ambulance","1":"Swiss Air Ambulance","2":"Swiss Air Ambulance"},"UNIQUE_CARRIER_ENTITY":{"0":"71084","1":"71084","2":"71084"},"REGION":{"0":"I","1":"I","2":"I"},"CARRIER":{"0":"10Q","1":"10Q","2":"10Q"},"CARRIER_NAME":{"0":"Swiss Air Ambulance","1":"Swiss Air Ambulance","2":"Swiss Air Ambulance"},"CARRIER_GROUP":{"0":0,"1":0,"2":0},"CARRIER_GROUP_NEW":{"0":0.0,"1":0.0,"2":0.0},"ORIGIN_AIRPORT_ID":{"0":11618,"1":12651,"2":12889},"ORIGIN_AIRPORT_S

# Reading Excel

In [120]:
df2 = pd.read_excel('out.xls')
df2

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,...,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50
37,0,61,5365411,18353,16933,571685,0,4395,0,0,...,6,627,1,2015,1,1,9,F,IF,NaN
42,0,61,5365411,18353,16827,1274568,0,4395,0,0,...,6,627,1,2015,1,1,9,F,IF,NaN
158,0,51,2706103,17384,13517,0,0,2002,0,0,...,6,627,1,2015,1,1,5,F,IF,NaN
159,0,51,2751949,17384,13746,0,0,2002,0,0,...,6,627,1,2015,1,1,5,F,IF,NaN
181,0,62,9292527,20584,16739,1505486,0,6504,0,0,...,6,637,1,2015,1,1,14,F,IF,NaN
186,0,62,9256991,20584,19123,1847391,0,6504,0,0,...,6,637,1,2015,1,1,14,F,IF,NaN
199,0,184,25907200,0,0,23870728,0,1506,0,0,...,6,696,2,2015,1,1,4,G,IF,NaN
201,0,62,8729600,0,0,7389870,0,1388,0,0,...,6,696,2,2015,1,1,3,G,IF,NaN
205,0,134,18867200,0,0,9486624,0,1506,0,0,...,6,696,2,2015,1,1,4,G,IF,NaN
212,0,59,4517040,17818,14384,309318,0,4661,0,0,...,6,696,1,2015,1,1,10,F,IF,NaN


In [121]:
! wget 'http://datos.madrid.es/egob/catalogo/207831-0-accidentes-trafico.xls'

--2019-01-12 12:48:51--  http://datos.madrid.es/egob/catalogo/207831-0-accidentes-trafico.xls
Resolving datos.madrid.es (datos.madrid.es)... 149.235.17.74
Connecting to datos.madrid.es (datos.madrid.es)|149.235.17.74|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://datos.madrid.es/egob/catalogo/207831-0-accidentes-trafico.xls [following]
--2019-01-12 12:48:54--  https://datos.madrid.es/egob/catalogo/207831-0-accidentes-trafico.xls
Connecting to datos.madrid.es (datos.madrid.es)|149.235.17.74|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://datos.madrid.es:443/egobfiles/MANUAL/207831/01_%20ACCIDENTES%20POR%20TIPO%20EN%20%20DISTRITOS.xls [following]
--2019-01-12 12:48:54--  https://datos.madrid.es/egobfiles/MANUAL/207831/01_%20ACCIDENTES%20POR%20TIPO%20EN%20%20DISTRITOS.xls
Connecting to datos.madrid.es (datos.madrid.es)|149.235.17.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61952 

In [169]:
pd.read_excel?

In [174]:
all_accidents = pd.read_excel('207831-0-accidentes-trafico.xls', index_col=0,usecols='A:K', header=7, sheet_name=None, skip_footer=1)
all_accidents['2013']

,366,40,143,98,3,31,8,12,5,3.1
CARABANCHEL,,,,,,,,,,
CENTRO,482,33,159,138,1.0,80,9,12,10.0,2.0
CHAMARTIN,451,82,121,86,4.0,84,5,2,5.0,4.0
CHAMBERI,414,30,67,91,1.0,87,9,7,4.0,5.0
CIUDAD LINEAL,396,52,128,75,3.0,72,10,6,4.0,4.0
FUENCARRAL-EL PARDO,335,42,159,72,9.0,39,7,13,2.0,4.0
HORTALEZA,171,18,77,51,3.0,33,8,10,1.0,2.0
LATINA,248,29,110,95,4.0,24,4,4,5.0,7.0
MONCLOA-ARAVACA,300,65,168,59,12.0,68,4,13,9.0,4.0
MORATALAZ,132,14,39,53,7.0,23,5,5,1.0,3.0


In [175]:
for k, v in all_accidents.items():
    print(v)

                                COLISIÓN DOBLE                            \
DISTRITO_ACCIDENTE                                                         
ARGANZUELA                                                           389   
BARAJAS                                                               89   
CARABANCHEL                                                          375   
CENTRO                                                               514   
CHAMARTIN                                                            494   
CHAMBERI                                                             414   
CIUDAD LINEAL                                                        398   
FUENCARRAL-EL PARDO                                                  321   
HORTALEZA                                                            215   
LATINA                                                               298   
MONCLOA-ARAVACA                                                      340   
MORATALAZ   

#### Exercise (homework)

Consolidate the excel into one DataFrame: you will need to create a 'YEAR' column. Think how you can iterate through all the DataFrames. Think also where you can take the value of 'YEAR' from.

In [184]:
for año, dataframe in all_accidents.items():
    dataframe['AÑO'] = año


In [196]:
solo_hasta_2012=  filter(lambda frame : int(frame["AÑO"][0]) < 2013,   all_accidents.values())
concatenated = pd.concat(solo_hasta_2012 )

In [197]:
concatenated

,COLISIÓN DOBLE,COLISIÓN MÚLTIPLE,CHOQUE CON OBJETO FIJO,ATROPELLO,VUELCO,CAÍDA MOTOCICLETA,CAÍDA CICLOMOTOR,CAÍDA BICICLETA,CAÍDA VIAJERO BUS,OTRAS CAUSAS,AÑO
DISTRITO_ACCIDENTE,,,,,,,,,,,
ARGANZUELA,389,54,126,75,9.0,43,9.0,4.0,5.0,8.0,2009
BARAJAS,89,6,53,21,NaN,12,2.0,1.0,2.0,2.0,2009
CARABANCHEL,375,44,171,137,8.0,36,13.0,6.0,9.0,8.0,2009
CENTRO,514,55,171,143,NaN,61,12.0,3.0,10.0,12.0,2009
CHAMARTIN,494,70,133,92,10.0,72,22.0,2.0,6.0,7.0,2009
CHAMBERI,414,35,87,87,3.0,51,12.0,4.0,4.0,5.0,2009
CIUDAD LINEAL,398,64,124,101,3.0,57,17.0,5.0,4.0,4.0,2009
FUENCARRAL-EL PARDO,321,47,151,78,11.0,34,16.0,7.0,2.0,8.0,2009
HORTALEZA,215,16,109,58,3.0,29,13.0,7.0,4.0,4.0,2009


Como 

# Reading mysql database

In [198]:
pd.read_sql_query('SELECT * FROM traffic WHERE SEATS > 160', conn)

,index,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,...,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50
0,14,0.0,9.0,562962.0,2548.0,2154.0,0.0,0.0,2188.0,0.0,...,6,696,1,2015,1,1,5,F,IF,None
1,15,0.0,18.0,1237995.0,5159.0,4905.0,0.0,0.0,1873.0,0.0,...,6,696,1,2015,1,1,4,F,IF,None
2,16,0.0,9.0,565173.0,2595.0,2156.0,0.0,0.0,2188.0,0.0,...,6,696,1,2015,1,1,5,F,IF,None
3,17,0.0,18.0,1237436.0,5109.0,4855.0,0.0,0.0,1873.0,0.0,...,6,696,1,2015,1,1,4,F,IF,None
4,18,0.0,4.0,404800.0,1260.0,558.0,0.0,0.0,5713.0,0.0,...,6,696,1,2015,1,1,12,F,IF,None
5,19,0.0,9.0,910800.0,2835.0,1666.0,2882.0,0.0,4879.0,0.0,...,6,696,1,2015,1,1,10,F,IF,None
6,20,0.0,4.0,404800.0,1260.0,833.0,0.0,0.0,5713.0,0.0,...,6,696,1,2015,1,1,12,F,IF,None
7,21,0.0,9.0,910800.0,2835.0,1758.0,39954.0,0.0,4879.0,0.0,...,6,696,1,2015,1,1,10,F,IF,None
8,22,0.0,23.0,2345933.0,6793.0,4946.0,454689.0,0.0,3749.0,0.0,...,6,696,1,2015,1,1,8,F,IF,None
9,23,0.0,29.0,2557427.0,8525.0,7094.0,621786.0,0.0,4733.0,0.0,...,6,696,1,2015,1,1,10,F,IF,None
